# 1. Configuración de entorno

En esta sección validamos que nuestro entorno de trabajo esté correctamente configurado antes de comenzar el análisis.  
Los pasos incluyen:

1. **Versión de Python**  
   - Se verifica que esté instalada la versión **3.11 o superior** (se recomienda 3.13).  
   - Esto garantiza compatibilidad con librerías modernas de análisis de datos y machine learning.

2. **Importación de librerías base**  
   - Se cargan librerías fundamentales:  
     - `numpy`, `pandas`: manipulación y análisis de datos.  
     - `matplotlib`, `seaborn`: visualización de datos.  
     - `scipy`: funciones estadísticas.  
   - Además se configuran estilos gráficos y opciones de visualización en pandas para trabajar con tablas más grandes.

3. **Verificación de versiones críticas**  
   - Se comprueba que `scikit-learn` esté instalado y en una versión **>= 1.0.1**.  
   - Esto es esencial ya que `scikit-learn` se usará para el modelado (baseline y posteriores).

Con esta configuración inicial aseguramos que el entorno sea reproducible y que todas las dependencias necesarias estén listas antes de continuar con el **EDA** y el **baseline**.


In [ ]:
import sys
import warnings
warnings.filterwarnings('ignore')

assert sys.version_info >= (3, 11), "Este notebook trabajo con python 3.11 o superiores (recomendado 3.13)"

print(f"Python {sys.version_info.major}.{sys.version_info.minor} instalado correctamente")

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy import stats

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12
sns.set_palette("husl")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', '{:.2f}'.format)

print("Librerías importadas exitosamente")

In [ ]:
# Verificar versiones de librerías críticas
from packaging import version
import sklearn

assert version.parse(sklearn.__version__) >= version.parse("1.0.1"), "Requiere scikit-learn >= 1.0.1"
print(f"scikit-learn {sklearn.__version__} instalado")

# 2. Metodología CRISP-DM
## 2.1. Comprensión del Negocio
El problema de Google Play Store  

**Contexto:**  
Es 2025. El mercado de aplicaciones móviles es altamente competitivo: millones de apps conviven en Google Play Store.  
Los desarrolladores buscan mejorar la visibilidad de sus aplicaciones y los usuarios dependen del **rating promedio** para decidir qué descargar.  

**Problema actual:**  
- El rating se conoce **solo después** de que los usuarios descargan y reseñan.  
- Las valoraciones son **altamente variables** y pueden depender de múltiples factores (categoría, descargas, precio, tamaño, tipo de app).  
- Los desarrolladores carecen de una herramienta para **estimar la calificación potencial** de una app antes o durante su lanzamiento.  
- La competencia es muy alta: una diferencia de décimas en rating puede significar miles de descargas menos.  

---

### 2.1.1. Solución propuesta  
Construir un **sistema automático de predicción de rating** de apps a partir de sus características disponibles en el dataset de Google Play Store.  

---

### 2.1.2. Definiendo el éxito  

**Métrica de negocio:**  
- Ayudar a los desarrolladores a anticipar la valoración probable de su app.  
- Reducir la dependencia de pruebas de mercado costosas o lentas.  
- Identificar características clave que favorecen una alta valoración (≥ 4.3).  

**Métrica técnica:**  
- Lograr un **Error Absoluto Medio (MAE) < 0.5 estrellas** en la predicción de rating.  
- Para la versión de clasificación (alta vs. baja calificación): obtener un **F1-score > 0.70**.  

**¿Por qué estos valores?**  
- El rating va de 1 a 5 → un error de 0.5 equivale a 10% de la escala.  
- Una diferencia de medio punto puede marcar la visibilidad de la app en el ranking.  
- Tasadores humanos (usuarios) también muestran variabilidad similar en sus calificaciones.  

---

### 2.1.3 Preguntas críticas antes de empezar  

1. **¿Realmente necesitamos ML?**  
   - Alternativa 1: Calcular el promedio de ratings por categoría → demasiado simple, no captura variabilidad.  
   - Alternativa 2: Reglas heurísticas (ej. “si es gratis y tiene muchas descargas, tendrá rating alto”) → insuficiente.  
   - **Conclusión:** Sí, ML es apropiado para capturar relaciones no lineales y múltiples factores.  

2. **¿Qué pasa si el modelo falla?**  
   - Transparencia: aclarar que es una estimación automática.  
   - Complementar con rangos de predicción (ej: intervalo de confianza).  
   - Mantener como referencia comparativa, no como único criterio de éxito.  

3. **¿Cómo mediremos el impacto?**  
   - Capacidad de anticipar apps con alta probabilidad de éxito.  
   - Ahorro de tiempo en validaciones preliminares.  
   - Insights para desarrolladores sobre qué factores influyen más en el rating.  

---


## 2.2. Comprensión de los Datos  

El objetivo de esta fase es explorar y entender el dataset de Google Play Store antes de construir modelos **(análisis exploratorio)**.  
Nos centraremos en:  

1. **Vista rápida del dataset**  
   - Identificar dimensiones (filas × columnas).  
   - Tipos de datos (numéricos, categóricos, texto, fechas).  
   - Valores faltantes obvios y rangos sospechosos.

2. **Descripción de variables**  
   - Revisar cada columna y entender su significado.  
   - Detectar qué variables podrían ser útiles como predictores y cuál será la variable objetivo (rating).  

3. **Detección de problemas en los datos**  
   - Análisis de valores faltantes.  
   - Estrategias: eliminar filas/columnas, imputar valores o crear indicadores de “dato faltante”.  

4. **Estadísticas descriptivas y univariadas**  
   - Media vs mediana (sesgo de la distribución).  
   - Desviación estándar (variabilidad, posibles outliers).  
   - Mínimos/máximos sospechosos.  
   - Histogramas para ver forma (normal, sesgada, bimodal, uniforme, picos extraños).  

5. **Análisis de variables categóricas**  
   - Distribución de categorías (ej. categorías de apps, tipo de app, content rating).  
   - Detección de clases dominantes o categorías poco representadas.  

6. **Correlaciones y relaciones entre variables**  
   - Matriz de correlación de Pearson para variables numéricas.  
   - Identificar relaciones fuertes, moderadas o débiles.  
   - Importante: recordar que **correlación ≠ causalidad**.  
7. ** Análisis de outliers **
   -  Tipos e identificación de outliers a través de diferentes métodos.

---

**Nota:**  
No siempre es necesario aplicar todos los pasos con igual profundidad.  
- Para este proyecto, el foco está en **identificar variables relevantes para predecir el rating** y **limpiar datos inconsistentes**.  
- Otros análisis más complejos (ej. NLP sobre descripciones) se pueden dejar como trabajo futuro (según trabajos de referencia investigados).

---


### 2.2.1 Descarga de datos  

En este paso descargamos el dataset de Google Play Store desde Kaggle y lo organizamos en la estructura de carpetas del proyecto.  

1. Usamos la librería `kagglehub` para acceder al dataset público **`lava18/google-play-store-apps`** directamente desde Kaggle.  
2. Se define una ruta clara dentro del proyecto para almacenar los datos originales: `../data/original/google-play-store/`. Esto ayuda a mantener la reproducibilidad y una estructura organizada.  
3. Con la función `shutil.copytree` copiamos los archivos descargados a la carpeta destino. De esta forma, el dataset queda disponible en nuestro directorio de trabajo para su análisis posterior.  


In [ ]:
import kagglehub
import shutil

def download_data(origin_repository, target_folder):
    # Descargar dataset
    path = kagglehub.dataset_download(origin_repository)
    
    # Copiar los archivos descargados
    shutil.copytree(path, target_folder, dirs_exist_ok=True)
    

download_data("lava18/google-play-store-apps", "../data/original/google-play-store")

### 2.2.2 Carga de datos  

En este paso realizamos la **lectura del archivo CSV** que contiene el dataset descargado previamente.  

- Definimos una función `load_data(path, file)` que recibe la ruta y el nombre del archivo, y lo carga con `pandas.read_csv()`.  
- Cargamos el dataset principal en la variable `applications_data` desde la carpeta `../data/original/google-play-store/`.  
- Incluimos una verificación simple:  
  - Si el dataset se carga con éxito, se imprime `"Dataset loaded"`.  
  - En caso contrario, se muestra un mensaje de error.  

Con esta validación aseguramos que el archivo esté disponible y correctamente leído antes de continuar con el análisis exploratorio.


In [ ]:
import pandas as pd
import numpy as np

def load_data(path, file):
    return pd.read_csv(f"{path}/{file}")

def convert_numeric_columns(df: pd.DataFrame) -> pd.DataFrame:
    """
    Convierte a numéricas solo las columnas que deberían serlo, sin tocar 'Size'.
    Usa to_numeric(errors='coerce') para evitar ValueError si aparece texto.
    """
    df = df.copy()

    # Rating
    if "Rating" in df.columns:
        df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce")

    # Reviews: quitar comas y cualquier carácter no numérico/punto
    if "Reviews" in df.columns:
        df["Reviews"] = (
            df["Reviews"].astype(str)
            .str.replace(r"[^\d.]", "", regex=True)
            .pipe(pd.to_numeric, errors="coerce")
        )

    # Installs: quitar +, comas y cualquier carácter no numérico/punto
    if "Installs" in df.columns:
        df["Installs Numeric"] = (
            df["Installs"].astype(str)
            .str.replace(r"[^\d.]", "", regex=True)
            .pipe(pd.to_numeric, errors="coerce")
        )

    # Price: quitar $ y cualquier carácter no numérico/punto
    if "Price" in df.columns:
        df["Price"] = (
            df["Price"].astype(str)
            .str.replace(r"[^\d.]", "", regex=True)
            .pipe(pd.to_numeric, errors="coerce")
        )


    if "Size" in df.columns:
        def parse_size(x):
            if isinstance(x, str):
                x = x.strip()
                if x.endswith("M"):
                    return float(x[:-1])
                elif x.endswith("k") or x.endswith("K"):
                    return float(x[:-1]) / 1024  # KB -> MB
                else:
                    return np.nan
            return np.nan
        df["Size"] = df["Size"].apply(parse_size)

    return df


temp_applications_data = load_data("../data/original/google-play-store", "googleplaystore.csv")
applications_data = convert_numeric_columns(temp_applications_data)


if len(applications_data):
    print("Dataset cargado")
else:
    print("Error cargando dataset")


### 2.2.3 Vista rápida del dataset

**Dimensiones y columnas**
- Registros: **10,841** filas.
- Columnas: actualmente **14**; **originalmente eran 13** y se **añadió** una columna derivada: **`Installs Numeric`** para análisis con describe.
- Memoria aproximada: **~1.2 MB**.

**Tipos de datos (y transformaciones realizadas)**
- Numéricas (`float64`): `Rating`, `Reviews`, `Size`, `Price`, **`Installs Numeric`**.
- Categóricas / texto (`object`): `App`, `Category`, `Installs` *(forma original con “1,000+”)*, `Type`, `Content Rating`, `Genres`, `Last Updated`, `Current Ver`, `Android Ver`.
- Transformaciones ya aplicadas:
  - **`Installs`** se **conservó** en su formato original (categórico con “+” y comas) **y** se creó **`Installs Numeric`** mapeando esos rangos a números (0 … 1,000,000,000).
  - **`Price`**, **`Reviews`** y **`Size`** fueron normalizadas/parseadas a **numérico** para análisis y modelado.

**Valores faltantes (no-null count → faltantes aprox.)**
- `Rating`: 9,367 → **1,474 faltantes (~13.6%)**.
- `Size`: 9,145 → **1,696 faltantes (~15.6%)**.
- `Current Ver`: 10,833 → **8 faltantes (~0.07%)**.
- `Android Ver`: 10,838 → **3 faltantes (~0.03%)**.
- `Content Rating`: 10,840 → **1 faltante (~0.01%)**.
- `Price`: 10,840 → **1 faltante (~0.01%)**.
- `Installs Numeric`: 10,840 → **1 faltante (~0.01%)**.
- Resto de columnas: **sin faltantes**.

**Duplicados:**
-   Se identificaron **483 filas duplicadas** (≈ **4.46%** del
    dataset).\
-   Ejemplos de duplicados incluyen apps como:
    -   *Quick PDF Scanner + OCR FREE*\
    -   *Box*\
    -   *Google My Business*\
    -   *ZOOM Cloud Meetings*\
    -   *join.me -- Simple Meetings*\

**Rangos y valores sospechosos (según `describe()`)**
- `Rating`: **min = 1.0**, **max = 19.0** → **19** es inválido para la escala 1–5 (error de dato a corregir).
- `Reviews`: media ~ **444k**, **p75 ≈ 54,768**, **max ≈ 78M** → valores altos plausibles; tratar como **outliers**.
- `Size` (MB): media ~ **21.5**, **p50 = 13**, **p75 = 30**, **max = 100** → distribución sesgada a la derecha; mínimos muy bajos (**0.01**) a revisar.
- `Price` (USD): **mediana = 0** y **p75 = 0** → la mayoría son **apps gratuitas**; **max = 400** sugiere outliers de precio.
- `Installs Numeric`: **p25 = 1,000**, **p50 = 100,000**, **p75 = 5,000,000**, **max = 1,000,000,000** → escala muy amplia; conviene usar **transformaciones log** o **binning** en el EDA/modelado.

**Conclusión inicial**
- Los **faltantes** más relevantes están en `Rating` y `Size`; habrá que decidir estategía para aumentar, imputar o nivelar los datos.
- Existen **outliers (no legítimos)** (ej. `Rating = 19`) y variables con **colas largas** (ej. `Reviews`, `Installs Numeric`, `Price`).
- Eliminar **duplicados** para evitar sesgos de análisis y que no introduzcan ruidos.


In [ ]:
print("=" * 80)
print("INFORMACIÓN GENERAL DEL DATASET".center(80))
print("=" * 80)

display(applications_data.head().style.background_gradient(cmap='RdYlGn', subset=['Rating']))

# Información detallada
print("\n" + "=" * 80)
print("ESTRUCTURA DE DATOS".center(80))
print("=" * 80)
applications_data.info()

# Estadísticas descriptivas
print("\n" + "=" * 80)
print("ESTADÍSTICAS DESCRIPTIVAS".center(80))
print("=" * 80)
display(applications_data.describe().round(2).T)

print("\n" + "=" * 80)
print("DATOS DUPLICADOS".center(80))
print("=" * 80)

# Contar duplicados
num_duplicados = applications_data.duplicated().sum()
print(f"Total de registros duplicados: {num_duplicados}")

# Mostrar ejemplos de duplicados si existen
if num_duplicados > 0:
    print("\nEjemplos de filas duplicadas:\n")
    display(applications_data[applications_data.duplicated()].head())
else:
    print("No se encontraron registros duplicados.")


### 2.2.4 Descripción de las variables

In [ ]:
variables = [
    'App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price',
    'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver',
    'Installs Numeric'
]

tipos = [
    'Categórica',          # App
    'Categórica',          # Category
    'Numérica (Target)',   # Rating
    'Numérica',            # Reviews
    'Numérica (MB)',       # Size
    'Categórica (rango)',  # Installs
    'Categórica',          # Type
    'Numérica (USD)',      # Price
    'Categórica',          # Content Rating
    'Categórica',          # Genres
    'Texto (fecha)',       # Last Updated (parseable a fecha)
    'Texto',               # Current Ver
    'Texto',               # Android Ver
    'Numérica',            # Installs Numeric
]

descripciones = [
    'Nombre de la aplicación.',
    'Categoría oficial de la app en Google Play.',
    'Calificación promedio de usuarios (1 a 5).',
    'Número de reseñas reportadas.',
    'Tamaño aproximado de la app en MB.',
    'Instalaciones en rango (p.ej., "1,000+").',
    'Tipo de app (Free / Paid).',
    'Precio en USD (0 para gratuitas).',
    'Clasificación de contenido (Everyone, Teen, etc.).',
    'Género(s) de la app.',
    'Fecha de última actualización (texto en origen).',
    'Versión actual declarada por el desarrollador.',
    'Versión mínima de Android requerida.',
    'Instalaciones convertidas a número para análisis.'
]

valores_faltantes = [applications_data[col].isnull().sum() if col in applications_data.columns else None for col in variables]

metadata = {
    'Variable': variables,
    'Tipo': tipos,
    'Descripción': descripciones,
    'Valores Faltantes': valores_faltantes
}

df_metadata = pd.DataFrame(metadata)

# Mostrar con resaltado de faltantes
styled = df_metadata.style.applymap(
    lambda x: 'background-color: #ffcccc' if isinstance(x, (int, float)) and x > 0 else '',
    subset=['Valores Faltantes']
)

display(styled)

# Resumen de dtypes originales (informativo)
dtypes_resumen = applications_data[variables].dtypes.astype(str).reset_index()
dtypes_resumen.columns = ['Variable', 'dtype pandas']
display(dtypes_resumen)

### 2.2.5 Detección de problemas en los datos 

**Resumen de hallazgos (valores faltantes):**
- `Size` ≈ 15.6% y `Rating` ≈ 13.6% concentran la mayoría de los faltantes.
- Faltantes puntuales (≈0.01%): `Type`, `Price`, `Content Rating`, `Installs Numeric` ocurren en la misma(s) fila(s) → patrón conjunto.
- `Android Ver` (0.03%) y `Current Ver` (0.07%) con faltantes residuales, parcialmente correlacionados con el grupo anterior.

**Heatmap de correlación de patrones de faltantes (interpretación):**
- Correlación 1.00 entre `Type`, `Price`, `Content Rating`, `Installs Numeric`: las ausencias co-ocurren en el/los mismos registros. Acciones coordinadas.
- `Android Ver` muestra correlación moderada (~0.58) con ese grupo: algunas veces falta junto con ellos.
- `Size`, `Rating`, `Current Ver` tienen patrones de faltantes independientes del grupo anterior (correlaciones cercanas a 0), lo que sugiere causas distintas.

#### Posibles estrategias de corrección

- Limpieza básica
  - Eliminar duplicados (483 filas) para evitar sesgos.
  - Validar y corregir outliers imposibles, p. ej., `Rating = 19` → convertir a NaN para tratarlo como faltante.

- Imputación (conservadora y por grupos)
  - `Rating` (target): para modelado, eliminar filas sin `Rating`; para EDA descriptivo, imputar mediana por `Category` solo para visualización.
  - `Size`: imputar mediana por `Category × Type` y crear indicador `size_missing`.
  - `Android Ver`, `Current Ver`: imputar moda por `Category` y crear indicadores `androidver_missing`, `currentver_missing`.
  - Faltantes conjuntos (`Type`, `Price`, `Content Rating`, `Installs Numeric`):
    - Si es 1 fila: eliminarla es lo más simple y seguro.
    - Alternativa (si se prefiere imputar):
      - `Type`: inferir desde `Price` (0 → Free, >0 → Paid).
      - `Price`: 0 si `Type == Free`, si `Paid` usar mediana por `Category`.
      - `Content Rating`: moda por `Category`.
      - `Installs Numeric`: mediana por `Category × Type` o por bin de `Installs`.



#### Estrategias de “nivelación” según los porcentajes observados

- Size (~15.6% faltantes, >5% y <<60%)
  - Acción: imputar mediana por grupo `Category × Type`.
  - Añadir flag: `size_missing = 1` cuando falte (conserva señal de ausencia).
  - Justificación: volumen relevante; la mediana por grupos respeta diferencias entre tipos/categorías.

- Rating (~13.6% faltantes, >5% y <<60%) [variable objetivo]
  - Para modelado: eliminar filas sin `Rating` (evita sesgo por imputación del target).
  - Para EDA descriptivo: si se requiere visualizar completos, imputar mediana por `Category` solo para gráficos/tablas (no para entrenamiento).
  - Justificación: imputar el target puede distorsionar métricas.

- Current Ver (0.07%) y Android Ver (0.03%) (<5%)
  - Acción: imputar con la moda por `Category`. Flags opcionales `currentver_missing` y `androidver_missing`.
  - Justificación: impacto ínfimo; moda es suficiente y estable.

- Faltantes “en bloque” en la misma fila: Type, Price, Content Rating, Installs Numeric (≈0.01% cada uno; correlación 1.00)
  - Si es 1 fila: eliminarla directamente.
  - Si hubiera más en el futuro y se prefiriera imputar coordinadamente:
    - `Type` desde `Price` (0 → Free, >0 → Paid),
    - `Price` = 0 si `Free`, si `Paid` usar mediana por `Category`,
    - `Content Rating` = moda por `Category`,
    - `Installs Numeric` = mediana por `Category × Type`.
  - Justificación: co-ocurren; eliminar 1 fila no afecta el conjunto y evita inconsistencias.

- Transformaciones para estabilizar distribuciones (complementarias a la imputación)
  - `Reviews` y `Installs Numeric`: aplicar `log1p` para análisis y futuros modelos. *****************************
  - `Installs` (rangos): tratar como ordinal/bins en el EDA.

- Limpieza previa necesaria
  - Eliminar duplicados (483 filas).
  - Corregir valores imposibles detectados en el EDA (ej. `Rating = 19` → NaN) y re-entrar al flujo de imputación/nivelación anterior.

In [ ]:
def analyze_missing_values(df: pd.DataFrame) -> pd.DataFrame:
    """Análisis completo de valores faltantes con visualizaciones."""
    missing_counts = df.isnull().sum()
    missing_pct = (missing_counts / len(df)) * 100

    missing_df = pd.DataFrame({
        'Columna': df.columns,
        'Valores_Faltantes': missing_counts.values,
        'Porcentaje': missing_pct.values,
        'Tipo_Dato': df.dtypes.values
    })

    missing_df = missing_df[missing_df['Valores_Faltantes'] > 0].sort_values('Porcentaje', ascending=False)

    if len(missing_df) == 0:
        print("No hay valores faltantes en el dataset")
        return missing_df

    # Visualización: barras y correlación de patrones de faltantes
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))

    # Gráfico de barras de % faltantes
    ax1.bar(missing_df['Columna'], missing_df['Porcentaje'], color='coral')
    ax1.set_xlabel('Columna')
    ax1.set_ylabel('Porcentaje de Valores Faltantes (%)')
    ax1.set_title('Valores Faltantes por Columna')
    ax1.axhline(y=5, color='r', linestyle='--', label='Umbral 5%')
    ax1.axhline(y=60, color='purple', linestyle='--', label='Umbral 60%')
    ax1.tick_params(axis='x', rotation=90)
    ax1.legend()

    # Heatmap de correlación de patrones de faltantes
    mask_df = df[missing_df['Columna'].tolist()].isnull().astype(int)
    if mask_df.shape[1] >= 2:
        corr = mask_df.corr()
        sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm', vmin=-1, vmax=1, ax=ax2)
        ax2.set_title('Correlación de Patrones de Valores Faltantes')
    else:
        ax2.axis('off')
        ax2.set_title('Correlación de faltantes (no aplica: 1 columna)')

    plt.tight_layout()
    plt.show()

    return missing_df

missing_analysis = analyze_missing_values(applications_data)
if missing_analysis is not None and not missing_analysis.empty:
    display(missing_analysis)

### 2.2.6 Estadisticas descriptivas y univariadas (númerico)

A partir de la tabla de estadísticas y los gráficos generados para `Rating`, `Reviews`, `Size`, `Price` e `Installs Numeric`, se observan los siguientes puntos clave.

- Rating
  - Media ≈ 4.19 y mediana ≈ 4.30 → ligera cola a la izquierda (más apps con rating alto). Hay un valor imposible (≈19), confirmado en el boxplot/Q-Q como outlier extremo.
  - Outliers: ~5% por IQR, dominados por el valor inválido y algunos ratings bajos.
  - Q-Q plot: desviación frente a normalidad, esperable para una variable acotada [1,5].
  - Implicación/acción: eliminar filas sin `Rating` para modelado; corregir `Rating=19 → NaN` y excluir; no aplicar transformaciones (la escala es ya interpretables).

- Reviews
  - Media ≫ mediana (pico en 0–pocos miles; máximo ≈ 78M) → cola muy larga a la derecha.
  - Boxplot: ~18% outliers por IQR (muchas apps con reseñas muy altas).
  - Q-Q plot: gran desviación de normalidad (heavy tail).
  - Relación con Rating: correlación positiva muy débil (~0.07), tendencia casi plana.
  - Implicación/acción: usar `log1p(Reviews)` para estabilizar la distribución en análisis/modelado; considerar winsorizar p99.9 para vistas tabulares si se desea.

- Size (MB)
  - Media > mediana (≈ 21.5 vs 13) → sesgo a la derecha; valores hasta 100 MB.
  - ~6% outliers por IQR, especialmente en colas altas.
  - Q-Q plot: curvatura en colas; no normal.
  - Relación con Rating: correlación positiva débil (~0.08); señal muy tenue.
  - Implicación/acción: imputar faltantes por `Category × Type` y añadir `size_missing`; opcionalmente probar `log1p(Size)` o binning para robustecer.

- Price (USD)
  - Mediana = 0 (mayoría gratis) y cola a la derecha con máximos altos (≈ 400).
  - ~7% outliers por IQR; Q-Q muestra heavy tail.
  - Relación con Rating: correlación negativa muy débil (~-0.02).
  - Implicación/acción: crear `is_free = (Price == 0)` y, si se usa `Price` continuo, considerar `log1p(Price)` para las pocas apps pagas; validar coherencia `Type=Free ⇒ Price=0`.

- Installs Numeric
  - Media ≫ mediana (100k) con máximo 1e9 → distribución extremadamente sesgada a la derecha.
  - ~7–8% outliers por IQR; Q-Q muy alejado de normalidad.
  - Relación con Rating: correlación débil positiva (~0.05) y tendencia casi plana.
  - Implicación/acción: usar `log1p(Installs Numeric)` o bins ordinales para análisis; verificar coherencia con `Installs` textual.

Recomendaciones transversales
- Eliminar duplicados antes de resumir para evitar sesgos.
- Tratar outliers evidentes no-legítimos (p. ej. `Rating=19`). Para colas largas legítimas (`Reviews`, `Installs Numeric`, `Price`): preferir `log1p` o winsorización solo para visualizaciones.
- Mantener consistencia: `Type=Free ⇒ Price=0`; `Installs Numeric` coherente con el rango de `Installs`.
- Para relaciones con `Rating`, las correlaciones lineales observadas son débiles; la señal puede emerger mejor con interacciones (p. ej., `is_free × installs_bin`) o modelos no lineales.



In [ ]:
from scipy import stats

# Selección de columnas numéricas relevantes
numeric_cols = [c for c in ['Rating', 'Reviews', 'Size', 'Price', 'Installs Numeric'] if c in applications_data.columns]

# Tabla de estadísticas básicas (media, mediana, std, min, p25, p50, p75, max)
describe_tbl = applications_data[numeric_cols].describe(percentiles=[0.25, 0.5, 0.75]).T

# Métricas adicionales robustas
extra = pd.DataFrame(index=numeric_cols)
extra['mad'] = [stats.median_abs_deviation(applications_data[c].dropna()) for c in numeric_cols]
extra['skew'] = [applications_data[c].skew(skipna=True) for c in numeric_cols]
extra['kurtosis'] = [applications_data[c].kurtosis(skipna=True) for c in numeric_cols]
extra['cv'] = [applications_data[c].std(skipna=True) / applications_data[c].mean(skipna=True) if applications_data[c].mean(skipna=True) not in [0, np.nan] else np.nan for c in numeric_cols]

stats_table = describe_tbl.join(extra)
display(stats_table.round(3))


def univariate_analysis(df: pd.DataFrame, column: str, target: str | None = None):
    """Análisis univariado con histograma, boxplot, Q-Q plot y relación con target."""
    series = df[column].dropna()
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))

    # 1) Histograma con líneas de media y mediana
    ax1 = axes[0, 0]
    ax1.hist(series, bins=50, edgecolor='black', alpha=0.7)
    ax1.axvline(series.mean(), color='red', linestyle='--', label=f"Media: {series.mean():.2f}")
    ax1.axvline(series.median(), color='green', linestyle='--', label=f"Mediana: {series.median():.2f}")
    ax1.set_title(f"Distribución de {column}")
    ax1.set_xlabel(column)
    ax1.set_ylabel('Frecuencia')
    ax1.legend()
    ax1.grid(alpha=0.3)

    # 2) Boxplot + conteo de outliers (IQR)
    ax2 = axes[0, 1]
    bp = ax2.boxplot(series, vert=True, patch_artist=True)
    bp['boxes'][0].set_facecolor('lightblue')
    Q1, Q3 = series.quantile(0.25), series.quantile(0.75)
    IQR = Q3 - Q1
    outliers_mask = (series < Q1 - 1.5 * IQR) | (series > Q3 + 1.5 * IQR)
    n_out = int(outliers_mask.sum())
    pct_out = 100 * n_out / len(series) if len(series) else 0
    ax2.set_title(f"Boxplot de {column}")
    ax2.set_ylabel(column)
    ax2.grid(alpha=0.3)
    ax2.text(1.1, Q3, f"Outliers: {n_out} ({pct_out:.1f}%)", fontsize=10)

    # 3) Q-Q plot normal
    ax3 = axes[1, 0]
    stats.probplot(series, dist='norm', plot=ax3)
    ax3.set_title('Q-Q Plot (Normalidad)')
    ax3.grid(alpha=0.3)

    # 4) Relación con target si aplica
    ax4 = axes[1, 1]
    if target is not None and target in df.columns and column != target:
        valid = df[[column, target]].dropna()
        ax4.scatter(valid[column], valid[target], alpha=0.4, s=10)
        ax4.set_xlabel(column)
        ax4.set_ylabel(target)
        ax4.set_title(f"{column} vs {target}")
        # Línea de tendencia (ajuste lineal simple)
        if len(valid) > 1:
            z = np.polyfit(valid[column], valid[target], 1)
            p = np.poly1d(z)
            xs = np.linspace(valid[column].min(), valid[column].max(), 200)
            ax4.plot(xs, p(xs), 'r--', alpha=0.8, label='Tendencia')
            corr = valid[column].corr(valid[target])
            ax4.text(0.05, 0.95, f"Correlación: {corr:.3f}", transform=ax4.transAxes,
                     fontsize=10, bbox=dict(boxstyle='round', facecolor='wheat'))
            ax4.legend()
    else:
        ax4.axis('off')
        ax4.grid(alpha=0.3)

    plt.suptitle(f"Análisis Univariado: {column}", fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Ejecutar análisis univariado para cada métrica numérica, relacionando con Rating
for col in numeric_cols:
    univariate_analysis(applications_data, col, target='Rating')

### 2.2.7. Análisis Univariado Categórico
- Category
  - Distribución: alta concentración en `FAMILY` (~19%) y `GAME` (~12%). El resto de categorías tienen menor peso individual; el grupo `Others` acumula ~31% del total.
  - Rating por categoría: diferencias moderadas; la **mediana** suele estar entre 4.2–4.4. Algunas categorías muestran desviación estándar mayor (p. ej., `PRODUCTIVITY`, `LIFESTYLE`), indicando más variabilidad de valoración.
  - Implicaciones: riesgo de sesgo por categorías mayoritarias en análisis agregados. Para modelado, conviene usar dummies Top-K o codificación ordinal/target encoding con cuidado (evitar fuga). Agrupar colas largas en `Others` es adecuado para visualización.

- Content Rating
  - Distribución: `Everyone` domina (~79%), seguido por `Teen` (~12%); `Mature 17+` y `Everyone 10+` suman ~9% en conjunto; clases raras casi nulas.
  - Rating por nivel de contenido: medias similares (≈4.1–4.3). `Teen` tiende a mediana 4.3 y variabilidad algo menor; `Mature 17+` muestra algo más de dispersión.
  - Implicaciones: por el fuerte desbalance, esta variable aporta señal limitada por sí sola. Útil como interacción con `Category`/`Genres`.

- Type
  - Distribución: `Free` ≈ 93%, `Paid` ≈ 7% (clase muy desbalanceada); existe un registro anómalo (valor 0) en los gráficos que debe eliminarse/corregirse.
  - Rating por tipo: medias muy cercanas (Free ≈ 4.19, Paid ≈ 4.27). La diferencia es pequeña y probablemente no significativa sin controlar otras variables (p. ej., `Category`).
  - Implicaciones: por el desbalance extremo, conviene usar `is_free` como binaria y, si se modela interacción con `Installs` o `Price`, puede emerger señal. Validar regla `Type=Free ⇒ Price=0`.

- Genres Main (primer género)
  - Distribución: gran cola larga; `Others` concentra ~48%. Entre Top-12, `Tools`, `Entertainment` y `Education` destacan en frecuencia.
  - Rating por género: diferencias pequeñas (medianas ~4.2–4.4), con algunas variaciones en dispersión (p. ej., `Medical` y `Lifestyle` más variables).
  - Implicaciones: por la alta cardinalidad y colas largas, mantener Top-K + `Others` en EDA ayuda a la legibilidad. Para modelado, preferir codificación que reduzca dimensionalidad (Top-K dummies, hashing, o target encoding con validación adecuada).

In [ ]:
def analyze_categorical_compact(df: pd.DataFrame, cat_col: str, target_col: str, top_n: int = 12):
    """
    Versión compacta para variables con muchas categorías:
    - Ordena por frecuencia, muestra top_n y agrupa el resto en "Others".
    - Barras horizontales, pie chart compacto, boxplot y tabla para top_n.
    """
    data = df[[cat_col, target_col]].dropna(subset=[cat_col, target_col]).copy()
    if data.empty:
        print(f"Sin datos para {cat_col} y {target_col}")
        return

    counts = data[cat_col].value_counts()
    top_cats = counts.head(top_n)
    others_count = counts.iloc[top_n:].sum()

    # Mapeo a top_n + Others
    mapping = {c: c for c in top_cats.index}
    data['__cat__'] = data[cat_col].where(data[cat_col].isin(top_cats.index), other='Others')

    # Recalcular conteos con Others
    counts_compact = data['__cat__'].value_counts()
    order = list(top_cats.index) + (['Others'] if 'Others' in counts_compact.index else [])

    fig, axes = plt.subplots(2, 2, figsize=(14, 10))

    # 1) Barras horizontales (mejor legibilidad)
    ax1 = axes[0, 0]
    vals = counts_compact.loc[order]
    ax1.barh(range(len(vals)), vals.values, color=plt.cm.Set3(range(len(vals))))
    ax1.set_yticks(range(len(vals)))
    ax1.set_yticklabels(order)
    ax1.invert_yaxis()
    ax1.set_title(f'Distribución (Top {top_n}) de {cat_col}')
    ax1.set_xlabel('Frecuencia')
    for i, v in enumerate(vals.values):
        ax1.text(v, i, f'  {v} ({v/len(data)*100:.1f}%)', va='center')

    # 2) Pie chart compacto
    ax2 = axes[0, 1]
    ax2.pie(vals.values, labels=order, autopct='%1.1f%%', startangle=140,
            colors=plt.cm.Set3(range(len(vals))))
    ax2.set_title(f'Proporción (Top {top_n} + Others) de {cat_col}')

    # 3) Boxplot del target por categoría (solo top_n)
    ax3 = axes[1, 0]
    top_mask = data['__cat__'] != 'Others'
    data_top = data[top_mask]
    data_top.boxplot(column=target_col, by='__cat__', ax=ax3)
    ax3.set_title(f'{target_col} por {cat_col} (Top {top_n})')
    ax3.set_xlabel(cat_col)
    ax3.set_ylabel(target_col)
    plt.sca(ax3)
    plt.xticks(rotation=30, ha='right')

    # 4) Tabla de estadísticas por categoría (solo top_n y Others si existe)
    ax4 = axes[1, 1]
    ax4.axis('off')
    stats_by_cat = data.groupby('__cat__')[target_col].agg(['count', 'mean', 'median', 'std']).loc[order].round(2)
    table = ax4.table(cellText=stats_by_cat.reset_index().values,
                      colLabels=['Categoría', 'N', 'Media', 'Mediana', 'Desv.Est.'],
                      cellLoc='center', loc='center', colWidths=[0.35, 0.12, 0.16, 0.16, 0.16])
    table.auto_set_font_size(False)
    table.set_fontsize(9)
    table.scale(1.05, 1.25)
    for i in range(5):
        table[(0, i)].set_facecolor('#40E0D0')
        table[(0, i)].set_text_props(weight='bold')

    plt.suptitle(f'Análisis Categórico Compacto: {cat_col}', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Ejecutar la versión compacta para las categóricas clave
for cat in [c for c in ['Category', 'Content Rating', 'Type', 'Genres Main', 'Installs'] if c in applications_data.columns]:
    analyze_categorical_compact(applications_data, cat, 'Rating', top_n=12)

### 2.2.8. Análisis de correlación entre variables

#### 2.2.8.1. Variables con Mayor Relación
- Existe una fuerte correlación positiva entre **Installs Numeric** y **Reviews**:
  - Pearson: 0.64 (relación lineal moderada-fuerte).
  - Spearman: 0.97 (relación monótonica muy fuerte).
- Esto implica que a mayor número de instalaciones, mayor número de reseñas.

#### 2.2.8.2. Correlación de Pearson
- En general, las correlaciones de Pearson muestran relaciones más débiles que Spearman, lo cual indica que las relaciones lineales no son tan marcadas.
- **Installs Numeric y Reviews** presentan la correlación lineal más alta (0.64), siendo moderada-fuerte.
- **Size y Reviews** muestran una correlación positiva baja/Débil (0.24).
- El resto de variables (Rating, Price) tienen correlaciones casi nulas con las demás, lo que refleja poca relación lineal.

#### 2.2.8.3. Correlación de Spearman
- **Installs Numeric y Reviews** tienen la correlación más fuerte (0.97).
- **Size** muestra correlación moderada con **Reviews** (0.37) y con **Installs Numeric** (0.35).
- **Price** presenta correlaciones negativas con **Reviews** (-0.17) e **Installs Numeric** (-0.24).

#### 2.2.8.4. Observaciones Clave
- El número de instalaciones y las reseñas son las variables más relacionadas, lo cual es lógico, ya que más usuarios generan más interacciones.
- El tamaño de la aplicación influye ligeramente en reseñas e instalaciones, pero no de forma determinante.
- El precio no solo carece de relación positiva, sino que parece tener un impacto negativo sobre la popularidad (menos instalaciones y reseñas).

#### 2.2.8.5. Conclusión
- **Installs Numeric** y **Reviews** son las métricas más críticas en el dataset de **Google Play Store**, ya que reflejan el éxito y la popularidad de la aplicación.
- **Size** es un factor secundario con cierta relación.

In [ ]:
# Análisis de correlación mejorado para el proyecto de Google Play Store
def correlation_analysis(df):
    """Análisis de correlación con múltiples métricas"""
    
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    
    fig, axes = plt.subplots(1, 2, figsize=(20, 6))
    
    # 1. Correlación de Pearson
    corr_pearson = df[numeric_cols].corr(method='pearson')
    mask = np.triu(np.ones_like(corr_pearson), k=1)
    sns.heatmap(corr_pearson, mask=mask, annot=True, fmt='.2f', 
               cmap='coolwarm', center=0, ax=axes[0],
               vmin=-1, vmax=1, cbar_kws={"shrink": 0.8})
    axes[0].set_title('Correlación de Pearson (Lineal)')
    
    # 2. Correlación de Spearman  
    corr_spearman = df[numeric_cols].corr(method='spearman')
    sns.heatmap(corr_spearman, mask=mask, annot=True, fmt='.2f',
               cmap='coolwarm', center=0, ax=axes[1],
               vmin=-1, vmax=1, cbar_kws={"shrink": 0.8})
    axes[1].set_title('Correlación de Spearman (Monotónica)')
    
    plt.suptitle('Análisis de Correlación Multi-métrica', 
                fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    # Tabla de correlaciones importantes
    print("\n🔗 Correlaciones Significativas:")
    print("=" * 50)
    for method, corr_matrix in zip(['Pearson', 'Spearman'], [corr_pearson, corr_spearman]):
        print(f"\n{method}:")
        significant_corr = corr_matrix[(abs(corr_matrix) > 0.3) & (corr_matrix != 1)].stack()
        for (var1, var2), corr in significant_corr.items():
            strength = "Fuerte" if abs(corr) > 0.5 else "Moderada" if abs(corr) > 0.3 else "Débil"
            direction = "Positiva" if corr > 0 else "Negativa"
            print(f"  • {var1} y {var2}: {corr:+.3f} ({strength} {direction})")
    
# Ejecutar el análisis de correlación
correlation_analysis(applications_data)

#### 2.2.9. Análisis de Outliers (IQR, Z-Score e Isolation Forest)
**Resumen cuantitativo**
- Total de registros analizados: **10,841**.
- Filas marcadas como outlier por método:
  - IQR: **3,489** filas (32.18%) → refleja colas largas especialmente en `Reviews`, `Installs Numeric`, `Price`.
  - Z-Score (> |3|): **654** filas (6.03%) → mucho más selectivo, captura extremos verdaderamente alejados tras estandarización.
  - Isolation Forest (contaminación=10%): **1,084** filas (10.0%) → patrón no lineal de anomalías combinadas.
- Consenso entre métodos:
  - Detectadas por los 3 métodos: **502** filas (casos altamente anómalos).
  - Detectadas exactamente por 2 métodos: **731** filas (anómalas consistentes, revisar antes de decidir acción).

**Variables más afectadas (IQR)**
- `Reviews`: **1,925** outliers → distribución extremadamente sesgada; valores muy altos representan apps masivas (probablemente legítimos).
- `Installs Numeric`: **828** outliers → escalas de descargas masivas (1e7–1e9).
- `Price`: **800** outliers → pocos productos de precio elevado (≥ p75 + 1.5·IQR); revisar si son apps premium legítimas.
- `Size`: **564** outliers → tamaños extremos (muy grandes o inusualmente pequeños).
- `Rating`: **504** outliers → incluye valores extremos bajos y el caso inválido (`Rating=19`).

**Interpretación y criterios**
- Muchos outliers provienen de fenómenos de cola larga típicos (popularidad extrema o modelo freemium/premium).
- No se recomienda eliminar masivamente outliers de `Reviews` o `Installs Numeric` sin antes transformar (`log1p`) o agrupar (binning), para no perder información sobre apps exitosas.
- El valor inválido `Rating=19` debe normalizarse a `NaN` y excluirse de modelado. Otros ratings muy bajos pueden mantenerse (aportan contraste).
- Outliers en `Price` podrían segmentarse: gratis (0), bajo costo (0 < p ≤ 10), premium (10 < p ≤ 50), ultra premium (>50).


**Conclusión**
El comportamiento extremo de `Reviews` e `Installs Numeric` refleja la naturaleza desigual del mercado (unas pocas apps concentran gran parte de la atención). Un manejo cuidadoso (transformaciones y flags) preservará información útil sin distorsionar el entrenamiento. Se prioriza limpieza puntual (ratings inválidos) sobre eliminación agresiva de outliers.

In [ ]:
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler

def detect_outliers(df):
    """Detección de outliers usando múltiples métodos"""
    
    numeric_df = df.select_dtypes(include=[np.number])
    
    # Método 1: IQR
    outliers_iqr = pd.DataFrame()
    for col in numeric_df.columns:
        Q1 = numeric_df[col].quantile(0.25)
        Q3 = numeric_df[col].quantile(0.75)
        IQR = Q3 - Q1
        outliers = ((numeric_df[col] < Q1 - 1.5 * IQR) | 
                   (numeric_df[col] > Q3 + 1.5 * IQR))
        outliers_iqr[col] = outliers
    
    # Método 2: Z-Score
    from scipy import stats
    z_scores = np.abs(stats.zscore(numeric_df.fillna(numeric_df.median())))
    outliers_zscore = (z_scores > 3)
    
    # Método 3: Isolation Forest
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(numeric_df.fillna(numeric_df.median()))
    iso_forest = IsolationForest(contamination=0.1, random_state=42)
    outliers_iso = iso_forest.fit_predict(scaled_data) == -1
    
    # Visualización
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Plot 1: Outliers por columna (IQR)
    ax1 = axes[0, 0]
    outlier_counts = outliers_iqr.sum()
    ax1.bar(range(len(outlier_counts)), outlier_counts.values)
    ax1.set_xticks(range(len(outlier_counts)))
    ax1.set_xticklabels(outlier_counts.index, rotation=45, ha='right')
    ax1.set_title('Outliers por Variable (Método IQR)')
    ax1.set_ylabel('Número de Outliers')
    
    # Plot 2: Distribución de outliers por método
    ax2 = axes[0, 1]
    methods_comparison = pd.DataFrame({
        'IQR': outliers_iqr.any(axis=1).sum(),
        'Z-Score': outliers_zscore.any(axis=1).sum(),
        'Isolation Forest': outliers_iso.sum()
    }, index=['Outliers'])
    methods_comparison.T.plot(kind='bar', ax=ax2, legend=False)
    ax2.set_title('Comparación de Métodos de Detección')
    ax2.set_ylabel('Número de Outliers Detectados')
    ax2.set_xlabel('Método')
    
    # Plot 3: Heatmap de outliers
    ax3 = axes[1, 0]
    sample_outliers = outliers_iqr.head(100)
    sns.heatmap(sample_outliers.T, cmap='RdYlBu_r', cbar=False, ax=ax3,
               yticklabels=True, xticklabels=False)
    ax3.set_title('Mapa de Outliers (Primeras 100 filas)')
    ax3.set_xlabel('Observaciones')
    
    # Plot 4: Resumen estadístico
    ax4 = axes[1, 1]
    ax4.axis('off')
    summary_text = f"""
    Resumen de Detección de Anomalías:
    
    • Total de observaciones: {len(df):,}
    • Outliers por IQR: {outliers_iqr.any(axis=1).sum():,} ({outliers_iqr.any(axis=1).sum()/len(df)*100:.1f}%)
    • Outliers por Z-Score: {outliers_zscore.any(axis=1).sum():,} ({outliers_zscore.any(axis=1).sum()/len(df)*100:.1f}%)
    • Outliers por Isolation Forest: {outliers_iso.sum():,} ({outliers_iso.sum()/len(df)*100:.1f}%)
    
    Variables más afectadas:
    {chr(10).join([f'  - {col}: {count:,} outliers' 
                   for col, count in outlier_counts.nlargest(3).items()])}
    
    Investigar outliers antes de eliminar. 
    Pueden contener información valiosa.
    """
    ax4.text(0.1, 0.5, summary_text, transform=ax4.transAxes,
            fontsize=11, verticalalignment='center',
            bbox=dict(boxstyle='round', facecolor='lightyellow'))
    
    plt.suptitle('Análisis de Outliers y Anomalías', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
    
    return outliers_iqr, outliers_zscore, outliers_iso

# Ejecutar la detección de outliers en el dataset de aplicaciones
outliers_iqr, outliers_zscore, outliers_iso = detect_outliers(applications_data)